# Training meta only


In [1]:
from datetime import datetime

import numpy as np

import os

import pandas as pd

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from time import time, gmtime, strftime

from final_models import create_model_meta

from utils.models.modelUtils import calcAccMetaModel

from utils.datagenUtils.datagenUtils import checkDataframe, convertRowToDictionary

from utils.datagenUtils.DataSeqMetaModel import DataSequenceMetaModel

from utils.callbacks.MyCallbacks import MyCallbacks

from utils.telegramUtils.telegram_bot import telegram_send_message

from utils.callbacks.callbackUtils import plotTimesPerEpoch

from utils.fileDirUtils.fileDirUtils import createDirIfNotExists

In [2]:
#Verbose settings:
verbose = False

TF_VERBOSE = 1 # 1 = Progress bar 2 = one line per epoch only!
TF_DETERMINISTIC_OPS = 1 # Makes everything also on GPU deterministic

# Classes:
NUM_CLASS = 2  # FAKE | NO FAKE

# Optimizer parameters:
# Adam
LEARNING_RATE = 1e-5
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-8
#optimizers:

optimizer = Adam(LEARNING_RATE, BETA_1, BETA_2, EPSILON)

# Network Settings:

#for training the whole model
GLOBAL_BATCH_SIZE = 1024 # 16 for all three
EPOCHS = 100

# Custom telegram send text 
CUSTOM_TEXT = f'Batch Size: {GLOBAL_BATCH_SIZE}, Epochs: {EPOCHS}, Optimizer: Adam, Learning Rate; {LEARNING_RATE}, Beta_1: {BETA_1}, Beta_2: {BETA_2}, Epsilon: {EPSILON}'


telegram_send_message(f'-----------------START-----------------')
print('START')
print(CUSTOM_TEXT)
telegram_send_message(CUSTOM_TEXT)



START
Batch Size: 1024, Epochs: 100, Optimizer: Adam, Learning Rate; 1e-05, Beta_1: 0.9, Beta_2: 0.999, Epsilon: 1e-08


In [3]:
pathToTextLabelFiles = '/home/armin/repos/FKD-Dataset/009_meta_label'

trainTextFile = os.path.join(pathToTextLabelFiles, "train_meta_label.csv")
testTextFile = os.path.join(pathToTextLabelFiles, "test_meta_label.csv")
valTextFile = os.path.join(pathToTextLabelFiles, "val_meta_label.csv")


df_train_whole = pd.read_csv(trainTextFile, header=0, sep='\t')
df_test_whole = pd.read_csv(testTextFile, header=0, sep='\t')
df_val_whole = pd.read_csv(valTextFile, header=0, sep='\t')

checkpointDir = '/home/armin/repos/FKD-Dataset/011_checkpoints/'


In [4]:
# Other settings:

# Time settings:
current_time = datetime.now().strftime("%Y-%m-%d_%H:%M")

#Checkpoint settings:
checkpoint_name = f'meta_only'

checkpointDir = os.path.join(checkpointDir, (checkpoint_name + '_' + current_time))

fileName="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filePath = os.path.join(checkpointDir, fileName)

In [5]:
# #backup original dataseq

# import os
# import pandas as pd
# import numpy as np
# from numpy import asarray
# from tensorflow.keras.utils import Sequence

# class DataSequenceMetaModel(Sequence):
#     'Generates data for Keras'
#     def __init__(self, df, batch_size):
#         self.df = df
#         self.batch_size = batch_size
#         self.on_epoch_end()
        
#         len_of_df = np.array(len(self.df))


#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(np.floor(len(self.df) / self.batch_size))

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]

#         # Slice df
#         df_batch = self.df.take(batch_indices)

#         # Generate data
#         meta_x, global_y = self.__data_generation(df_batch, batch_indices)

#         return [meta_x], global_y

#     def on_epoch_end(self):
#         'Updates indexes after each epoch -> Resetting them' 
#         self.indices = np.arange(len(self.df))
        
#     def get_image(self, path, img_size):
#         return load_img(path, target_size=[img_size[0], img_size[1]])

#     def __data_generation(self, df_slice, batch_indices):
#         batch_x = []
#         batch_y = []

#         for index, rowRaw in enumerate(df_slice.itertuples(), 1):
#             row = convertRowToDictionary(rowRaw, df_slice.columns, True)
#             batch_x.append([ row['author_enc'], row['score'], row['hasNanScore'], row['upvote_ratio'], row['hasNanUpvote'] , row['num_comments']])
# #             batch_x.append([row['author_enc'], row['score'], row['hasNanScore'], row['upvote_ratio'], row['hasNanUpvote'], row['num_comments']])
#             batch_y.append([row['2_way_label']])
#         meta_x = np.array(batch_x)
#         global_y = np.array(batch_y).astype(np.float32)
        
        

#         return (meta_x, global_y)


In [6]:
df_train_whole = checkDataframe(df_train_whole, GLOBAL_BATCH_SIZE)
df_test_whole = checkDataframe(df_test_whole, GLOBAL_BATCH_SIZE)
df_val_whole = checkDataframe(df_val_whole, GLOBAL_BATCH_SIZE)   

In [7]:
train_seq_whole = DataSequenceMetaModel(df_train_whole, GLOBAL_BATCH_SIZE)
test_seq_whole = DataSequenceMetaModel(df_test_whole, GLOBAL_BATCH_SIZE)
val_seq_whole = DataSequenceMetaModel(df_val_whole, GLOBAL_BATCH_SIZE)

STEP_SIZE_TRAIN = len(df_train_whole) // GLOBAL_BATCH_SIZE
STEP_SIZE_TEST = len(df_test_whole) // GLOBAL_BATCH_SIZE
STEP_SIZE_VAL = len(df_val_whole) // GLOBAL_BATCH_SIZE

In [8]:
tensorboardDir = os.path.join(checkpointDir, 'tensorboard')
callbacks_list = MyCallbacks(tensorboardDir, filePath, earlyStopping=True).createCheckpoints()

In [9]:
mirrored_strategy = tf.distribute.MirroredStrategy()


with mirrored_strategy.scope():
    
    model = create_model_meta(NUM_CLASS, 6) 
    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=optimizer,
              metrics=['accuracy'])

    history = model.fit(train_seq_whole,
                    steps_per_epoch=STEP_SIZE_TRAIN, epochs=EPOCHS, 
                    validation_data=val_seq_whole,
                    validation_steps=STEP_SIZE_VAL,  
                    callbacks=callbacks_list,
                    verbose = TF_VERBOSE
                   )
    
    

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Epoch 1/100
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorfl

In [10]:
plotTimesPerEpoch(callbacks_list)

It took so long to train on one epoch: [0, 0, 0, 0, 0] minutes


In [11]:
calcAccMetaModel(model, val_seq_whole, GLOBAL_BATCH_SIZE, 'Val')
calcAccMetaModel(model, test_seq_whole, GLOBAL_BATCH_SIZE, 'Test')

Val Accuracy is 0.6116193804824561
Test Accuracy is 0.6119620339912281


In [12]:
telegram_send_message(f'-----------------DONE-----------------')
print('done')

done
